In [21]:
import sys,os,importlib,gc, re, string,pickle
import xarray as xr
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

os.chdir('/home/peter/Projects/tc_emulator/results')

sys.path.append('../scripts')
import _weather_pattern_class; importlib.reload(_weather_pattern_class)

atl = _weather_pattern_class.weather_patterns(source='ERA5', working_directory='./')
atl.load_input('ERA5_VWS-MSLP_noTC3')
years = np.array(range(1982,2021))
atl.set_split(years=years)
nrows,ncols = 5,4
tag = 'SOM_pcaInit%sx%s_v1' % (nrows,ncols)
atl.define_plot_environment(pre_mapping='mapping_raw', clustering=tag, post_mapping='mapping_sammon_1982-2020', nrows=nrows, ncols=ncols)
atl.stats_TC(file='tracks/tracks_ibtracks.csv', overwrite=False)

{'SOM': <minisom.MiniSom object at 0x7f9c24a2ca90>}
.//ERA5_VWS-MSLP_noTC3/mapping_raw_1982-2020/SOM_pcaInit5x4_v1/mapping_sammon_1982-2020/grid_5x4


In [22]:
comp_names = {
    'gWeaLag2Weight' : '',
    'gWeaLag2' : 'equal weight',
    'gWea' : 'no lag',
    'gnnWeaSST' : 'NN weather + SST',
    'sLWeaNeigh' : '',
    'sLWea' : 'no neighbors',
    'sLAll' : 'random',
    'wS100nnQrSST' : '',
    'wS50nnQrSST' : '50NN qr sst',
    'wS100nnQrSSTrel' : 'rel. SSTs',
    'wS100nnQrSSTwindChBefore' : 'incl. intens. ch. before',
    'wS100nn' : '100 nn',
    'wS50nn' : '50 nn',
    'wS20nn' : '20 nn',
    'wS50nnBCpost' : 'nn BC',
    'wS50nnNoSSTBC' : 'nn BC',
    'wS100nnNoSST' : 'no SST',
    'wS100nnQrSSTnoHist' : 'no history',
    'wS100nnQrSSTnoWeather' : 'no weather',
    'g' : 'formation',
    'sL' : 'duration',
    'wS': 'intensification'
    }

def siggi(s):
    if np.isnan(s): return ''
    if s < 0.1: return '*'
    #if s < 0.1: return '*'
    return ''



In [23]:
# choose components
alphabet = iter(list(string.ascii_uppercase))
alphabet2 = iter(list(string.ascii_uppercase))
version = iter(range(1,100))
comps_todo = [
    {'g':'gWeaLag2Weight', 'sL':'sLWeaNeigh', 'wS':'wS100nnQrSST', 'Emu':'Emu0', 'name':'main','l':next(alphabet),'c':'c', 'v':'main', 'vc':''},
    ]
for i,g in enumerate(['gWea', 'gWeaLag2', 'gnnWeaSST']):
    comps_todo.append({'g':g, 'sL':'sLWeaNeigh', 'wS':'wS100nnQrSST', 'Emu':'Emu0', 'name':'formation: '+comp_names[g],'l':next(alphabet),'c':'m', 'v':'v%s' %(next(version)), 'vc':'vG%s' %(i)})
for i,sL in enumerate(['sLAll','sLWea']):
    comps_todo.append({'g':'gWeaLag2Weight', 'sL':sL, 'wS':'wS100nnQrSST', 'Emu':'Emu0', 'name':'duration: '+comp_names[sL],'l':next(alphabet),'c':'orange', 'v':'v%s' %(next(version)), 'vc':'vD%s' %(i)})
for i,wS in enumerate(['wS100nn','wS20nn','wS100nnNoSST','wS100nnQrSSTnoWeather','wS100nnQrSSTnoHist']):
    # for i,wS in enumerate(['wS20nn','wS50nn','wS100nn'][::-1]):
    comps_todo.append({'g':'gWeaLag2Weight', 'sL':'sLWeaNeigh', 'wS':wS, 'Emu':'Emu0', 'name':'intensification: '+comp_names[wS],'l':next(alphabet),'c':'r', 'v':'v%s' %(next(version)), 'vc':'vI%s' %(i)})


In [24]:
indicator_dict = {
    'genesis' : 'storm formations',
    'storm_days' : 'storm days',
    'wind' : 'max. daily wind speed',
    'stoMaxWind_vs_stoLen' : 'max. wind vs storm length',
    'wind_vs_stoD' : 'max. wind vs storm day',
    'dWind_vs_stoD' : 'change in max. wind vs storm day',
    'wind' : 'acc. daily max. wind speeds',
    'ACE' : 'ACE',
    'Hur' : 'hurricanes',
    'MajHur' : 'major hurricanes',
    'pearson_median' : 'Pearson coef. of median',
    'pearson_mean' : 'Pearson coef. of mean',
    'spearman_median' : 'Spearman coef. of median',
    'spearman_mean' : 'Spearman coef. of mean',
}

for q in [5,17,83,95]:
    indicator_dict[q] = '%sth percentile' %(q)


In [25]:
def print_table_surrounding(val,content, caption='', label=''):
    columns = r''''''
    for color in [dt['c'] for dt in comps_todo]:
        # if color == 'c':
        columns += r'''>{\columncolor{%s}}c|''' %(color)
        # else:
        #     columns += r'''c|'''
    out = r'''
\begin{table}[h!]
\begin{center}
\caption{%s}
\label{tab:%s}
\begin{tabular}{l||%s}
%s
\end{tabular}
\end{center}
\end{table}
''' %(caption,label,r'c|'*val.shape[1],content)
    return out
def print_table_header(val):
    return r'{\rotatebox[origin=c]{90}{version}} & ' +' & '.join([r'{\rotatebox[origin=c]{90}{%s}} ' %(indicator_dict[ind].replace(' ','\n')) for ind in val.indicator.values]) +' \\\ \\hline \\hline \n'

def print_subheader(val,subheader):
    return '\multicolumn{%i}{c}{%s} \\\\' %(len(val)+1,subheader) + '\\hline \\hline \n'

def print_table(val, sig=None, highlight=np.argmax, subheader=''):
    s = ''
    for v in val.emu.values:
        dt = [dt for dt in comps_todo if dt['v'] == v][0]
        line = r'\rowcolor{%s}' %(dt['c']) + '\n' + dt['name']
        for ind in val.indicator.values:
            tmp = str(val.loc[v,ind].values)
            if sig is not None:
                tmp += siggi(sig.loc[v,ind].values)
            line += ' & ' + tmp
        s += line + ' \\\\ \\hline \n'
    return s

def absmin(x):
    return np.argmin(np.abs(x))


In [66]:
si = open('../written_latex/WCD/submission2/SI/sections/sensitivity.tex', 'w')
s = r'''
\documentclass[float=false, crop=false]{standalone}
\usepackage[subpreambles=true]{standalone}
\usepackage{import}
\begin{document}
'''
si.write(s)



133

In [67]:
# Tab: RMSD
indicators = ['genesis','storm_days','Hur','MajHur','ACE']
Z = xr.DataArray(coords={'emu':[dt['v'] for dt in comps_todo], 'indicator':indicators}, dims=['emu','indicator'])
sig = Z.copy()
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    vali = pickle.load(open(atl._dir_lvl4 + '/emulator/xValid/'+tag+'/validation.pkl', 'rb'))
    for indicator in indicators:
        Z.loc[dt['v'],indicator] = vali[indicator]['RMSD']
s = print_table_header(Z)
s += print_table(Z.round(2),highlight=absmin, subheader=indicator_dict[indicator])
caption = r'''
Root mean squared deviation for different indicators and versions of the emulator. See table \ref{tab:versions} for more details on these versions.'''
s = print_table_surrounding(Z, s, caption, 'rmsd')
si.write(s)
# print(s)

1468

In [68]:
# Tab: correlation
indicators = ['genesis','storm_days','Hur','MajHur','ACE']
Z = xr.DataArray(coords={'emu':[dt['v'] for dt in comps_todo], 'indicator':indicators}, dims=['emu','indicator'])
sig = Z.copy()
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    vali = pickle.load(open(atl._dir_lvl4 + '/emulator/xValid/'+tag+'/validation.pkl', 'rb'))
    for indicator in indicators:
        Z.loc[dt['v'],indicator] = vali[indicator]['pearson_median']['coef']
s = print_table_header(Z)
s += print_table(Z.round(2), highlight=np.max, subheader=indicator_dict[indicator])
caption = r'''
Pearson correlation coefficients between observations and the median of 1000 simulations for different indicators and versions of the emulator. See table \ref{tab:versions} for more details on these versions.'''
s = print_table_surrounding(Z, s, caption, 'corr')
si.write(s)

1504

In [69]:
# Tab: trends
indicators = ['genesis','storm_days','Hur','MajHur','ACE']
Z = xr.DataArray(coords={'emu':[dt['v'] for dt in comps_todo], 'indicator':indicators}, dims=['emu','indicator'])
sig = Z.copy()
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    vali = pickle.load(open(atl._dir_lvl4 + '/emulator/xValid/'+tag+'/validation.pkl', 'rb'))
    for indicator in indicators:
        Z.loc[dt['v'],indicator] = vali[indicator]['trend_median']['slope']
        sig.loc[dt['v'],indicator] = vali[indicator]['trend_median']['pval']
s = print_table_header(Z)
s += print_table(Z.round(2), sig=sig, highlight=absmin, subheader=indicator_dict[indicator])
caption = r'''
Linear trends in residuals between observations and the median of 1000 simulations for different indicators and versions of the emulator. See table \ref{tab:versions} for more details on these versions.'''
s = print_table_surrounding(Z, s, caption, 'trends')
si.write(s)

1528

In [70]:
# Tab: trends vs SST
indicators = ['genesis','storm_days','Hur','MajHur','ACE']
Z = xr.DataArray(coords={'emu':[dt['v'] for dt in comps_todo], 'indicator':indicators}, dims=['emu','indicator'])
sig = Z.copy()
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    vali = pickle.load(open(atl._dir_lvl4 + '/emulator/xValid/'+tag+'/validation.pkl', 'rb'))
    for indicator in indicators:
        Z.loc[dt['v'],indicator] = vali[indicator]['trend_vs_SST_median']['slope']
        sig.loc[dt['v'],indicator] = vali[indicator]['trend_vs_SST_median']['pval']
s = print_table_header(Z)
s += print_table(Z.round(2), sig=sig, highlight=absmin, subheader=indicator_dict[indicator])
caption = r'''
Linear trends in residuals between observations and the median of 1000 simulations for different indicators and versions of the emulator. See table \ref{tab:versions} for more details on these versions.'''
s = print_table_surrounding(Z, s, caption, 'trends')
si.write(s)

1539

In [71]:
# Figs: correlation
for indicator in ['genesis','storm_days','Hur','MajHur','ACE']:
    s = r'''\begin{figure}[h!]''' + '\n'
    for dt in comps_todo:
        if (indicator == 'genesis' and 'formation' in dt['name']) or (indicator == 'storm_days' and 'intensification' not in dt['name']) or indicator not in ['genesis', 'storm_days'] or dt['name'] == 'main':
            tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
            s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/seasonal_N1000_%s_sens.png}''' %(tag,indicator) + '\n'
    caption = r'Simulations of seasonal ' + indicator_dict[indicator] + ' for different versions of the emulator. The black line shows observations. The cyan line shows the median of 1000 simulations for each year, while the lighter (darker) shading shows the 66\% (\%95) of simulations.'
    label = r'corr_%s' %(indicator)
    s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
    si.write(s)
    # print(s)

In [72]:
# Figs: residuals
for indicator in ['genesis','storm_days','Hur','MajHur','ACE']:
    s = r'''\begin{figure}[h!]''' + '\n'
    for dt in comps_todo:
        if (indicator == 'genesis' and 'formation' in dt['name']) or (indicator == 'storm_days' and 'intensification' not in dt['name']) or indicator not in ['genesis', 'storm_days'] or dt['name'] == 'main':
            tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
            s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/seasonal_N1000_%s_residuals_sens.png}''' %(tag,indicator) + '\n'
    caption = r'Residuals of simulations of seasonal ' + indicator_dict[indicator] + ' for different versions of the emulator. The black line shows observations. The cyan line shows the median of 1000 simulations for each year, while the lighter (darker) shading shows the 66\% (\%95) of simulations.'
    label = r'residuals_%s' %(indicator)
    s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
    si.write(s)

In [73]:
# Figs: residuals vs SST
for indicator in ['genesis','storm_days','Hur','MajHur','ACE']:
    s = r'''\begin{figure}[h!]''' + '\n'
    for dt in comps_todo:
        if (indicator == 'genesis' and 'formation' in dt['name']) or (indicator == 'storm_days' and 'intensification' not in dt['name']) or indicator not in ['genesis', 'storm_days'] or dt['name'] == 'main':
            tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
            s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/seasonal_N1000_%s_residuals_vs_SSTs_sens.png}''' %(tag,indicator) + '\n'
    caption = r'Residuals of simulations of seasonal ' + indicator_dict[indicator] + ' for different versions of the emulator. The black line shows observations. The cyan line shows the median of 1000 simulations for each year, while the lighter (darker) shading shows the 66\% (\%95) of simulations.'
    label = r'residuals_%s' %(indicator)
    s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
    si.write(s)

In [74]:
# Figs: hist storm duration
s = r'''\begin{figure}[h!]''' + '\n'
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    if dt['sL'] != 'sLWeaNeigh' or tag == 'gWeaLag2Weight_sLWeaNeigh_wS100nnQrSST_Emu0':
        s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/hist_duration_N1000_sens.png}''' %(tag) + '\n'
caption = r'''
Histogram of storm durations for observed storms (blue) and simulated storms (orange).
'''
label = r'hist_duration'
s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
si.write(s)

570

In [75]:
si.write(r'\end{document}')
si.close()

In [ ]:
# # Tab: versions
# s = r'version & formation & duration & intensification \\ \hline' + '\n'
# for dt in comps_todo:
#     tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
#     s += r'\rowcolor{%s}' %(dt['c']) + '\n' + dt['v']
#     for comp in ['g','sL','wS']:
#         s += ' & %s ' %(comp_names[dt[comp]])
#     s += '\\\\ \n'

# s = r'''
# \begin{table}[h!]
# \begin{center}
# \caption{%s}
# \label{tab:%s}
# \begin{tabular}{r||%s}
# %s
# \end{tabular}
# \end{center}
# \end{table}
# ''' %('Variations of emulator designs.\newline','versions','c|c|c',s)
# si.write(s)
# # print(s)

In [29]:
# Figs: intensity vs storm day
s = r'''\begin{figure}[h!]''' + '\n'
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    if dt['wS'] != 'wS100nnQrSST' or tag == 'gWeaLag2Weight_sLWeaNeigh_wS100nnQrSST_Emu0':
        s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/stoD_vs_wind_N1000_sens.png}''' %(tag) + '\n'
caption = r'''
Daily intensity of a storm against the storm day. Gray shadings indicate kernel density estimates from observations, while colored contour-lines show the simualtions.
'''
label = r'2D_wind_vs_day'
s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
si.write(s)

1082

In [30]:
# Figs: d intensity vs storm day
s = r'''\begin{figure}[h!]''' + '\n'
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    if dt['wS'] != 'wS100nnQrSST' or tag == 'gWeaLag2Weight_sLWeaNeigh_wS100nnQrSST_Emu0':
        s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/stoD_vs_dWind_N1000_sens.png}''' %(tag) + '\n'
caption = r'''
Daily intensity change  against the storm day. Gray shadings indicate kernel density estimates from observations, while colored contour-lines show the simualtions.
'''
label = r'2D_dWind_vs_day'
s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
si.write(s)

1086

In [37]:
# Figs: max intensity vs storm duration
s = r'''\begin{figure}[h!]''' + '\n'
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    if dt['wS'] != 'wS100nnQrSST' or tag == 'gWeaLag2Weight_sLWeaNeigh_wS100nnQrSST_Emu0':
        s += r'''\includegraphics[draft=false,width=0.32\linewidth]{emulator/xValid/%s/hstoMaxWind_vs_stoLen_N1000_sens.png}''' %(tag) + '\n'
caption = r'''
Maximal intensity of a storm against its duration. Gray shadings indicate kernel density estimates from observations, while colored contour-lines show the simualtions.
'''
label = r'2D_LMIvsStoLen'
s += r'\caption{%s}\label{fig:%s}' %(caption,label) + '\n' + r'\end{figure}' + '\n\n'
si.write(s)

1137